# 🧠 Análisis Avanzado de Ventas con Pandas

Este notebook incluye técnicas más avanzadas de análisis de datos, como detección de outliers, uso de `apply()` y funciones `lambda`, y análisis multivariado sobre el archivo de ventas.

In [2]:
!pip install matplotlib

   ---------------------------------------- 0.0/8.1 MB ? eta -:--:--
   --------- ------------------------------ 1.8/8.1 MB 12.6 MB/s eta 0:00:01
   ---------------------------- ----------- 5.8/8.1 MB 15.3 MB/s eta 0:00:01
   ------------------------------------ --- 7.3/8.1 MB 12.9 MB/s eta 0:00:01
   ---------------------------------------- 8.1/8.1 MB 12.6 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.2 MB ? eta -:--:--
   ---------------------------------------- 2.2/2.2 MB 18.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/7.0 MB ? eta -:--:--
   ------------------- -------------------- 3.4/7.0 MB 22.3 MB/s eta 0:00:01
   ---------------------------------------  6.8/7.0 MB 19.1 MB/s eta 0:00:01
   ---------------------------------------- 7.0/7.0 MB 17.2 MB/s eta 0:00:00

   ---------------------------------------- 0/7 [pyparsing]
   ----- ---------------------------------- 1/7 [pillow]
   ----- ---------------------------------- 1/7 [pillow]
  


[notice] A new release of pip is available: 25.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Cargar archivo CSV
df = pd.read_csv('ventas_prueba_pandas.csv', parse_dates=['Fecha'])
df

,Fecha,Producto,Cantidad,Precio,Ciudad,Cliente,Total
0,2025-07-30,Smartphone,6,513.19,Guadalajara,María Delia Blanco,3079.14
1,2025-07-22,Smartphone,3,747.18,Cancún,Lic. Carla Morales,2241.54
2,2025-05-11,Teclado,5,106.40,Ciudad de México,Diego Nadia Hidalgo,532.00
3,2025-06-06,Monitor,10,1477.37,Cancún,Adela Cortés Zamudio,14773.70
4,2025-04-23,Audífonos,6,1828.65,Ciudad de México,Ing. Leticia Quintero,10971.90
...,...,...,...,...,...,...,...
3995,2025-04-24,Impresora,6,1326.20,Monterrey,Jorge Monroy,7957.20
3996,2025-06-09,Audífonos,2,1059.17,Cancún,María Teresa Salinas,2118.34
3997,2025-06-03,Audífonos,5,1122.71,Guadalajara,Lic. Pamela Aparicio,5613.55
3998,2025-03-12,Cámara,1,1677.64,Toluca,Concepción Carrasco,1677.64


## 🔍 1. Detección de outliers con regla de 1.5 * IQR

In [5]:
# Calcular IQR para Total
Q1 = df['Total'].quantile(0.25)
Q3 = df['Total'].quantile(0.75)
IQR = Q3 - Q1
limite_superior = Q3 + 1.5 * IQR
limite_inferior = Q1 - 1.5 * IQR

# Filtrar outliers
outliers = df[(df['Total'] > limite_superior) | (df['Total'] < limite_inferior)]
print(f'Total de outliers detectados: {len(outliers)}')
outliers.head()

Total de outliers detectados: 46


,Fecha,Producto,Cantidad,Precio,Ciudad,Cliente,Total
62,2025-05-05,Tablet,10,1955.82,Querétaro,Jonás Nájera,19558.2
74,2025-05-02,Cámara,10,1828.36,Toluca,Cornelio Claudia Santacruz,18283.6
134,2025-01-18,Monitor,10,1829.39,Toluca,Benjamín Óliver Villanueva,18293.9
216,2024-09-13,Cámara,10,1879.58,Puebla,Rubén Serrato Maya,18795.8
279,2024-09-18,Tablet,10,1947.16,Querétaro,Rosario Javier Villaseñor Arce,19471.6


## 🛠️ 2. Crear nuevas variables con `apply()` y `lambda`

In [ ]:
# Categorizar monto total
df['Segmento'] = df['Total'].apply(lambda x: 'Bajo' if x < 500 else ('Medio' if x < 1000 else 'Alto'))
df['Dia'] = df['Fecha'].dt.day_name(locale='es')
df[['Total', 'Segmento', 'Dia']].head()

## 📊 3. Análisis multivariado: producto vs ciudad vs segmento

In [ ]:
pivot = pd.pivot_table(df, values='Total', index='Producto', columns='Ciudad', aggfunc='sum')
pivot.plot(kind='bar', figsize=(10,6), title='Total de Ventas por Producto y Ciudad')
plt.ylabel('USD')
plt.grid(True)
plt.tight_layout()
plt.show()

## 🧮 4. Ventas por día de la semana y segmento

In [ ]:
resumen_segmento = df.groupby(['Dia', 'Segmento'])['Total'].sum().unstack().fillna(0)
resumen_segmento = resumen_segmento.reindex(['lunes', 'martes', 'miércoles', 'jueves', 'viernes', 'sábado', 'domingo'])
resumen_segmento.plot(kind='bar', stacked=True, figsize=(10,6), title='Ventas por Día y Segmento')
plt.ylabel('USD')
plt.xticks(rotation=45)
plt.grid(True)
plt.tight_layout()
plt.show()